In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import glob
import os
import json
import cv2
import re
from sklearn.model_selection import train_test_split
import albumentations as A

In [3]:
# YOLO v5

!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils # yolo의 utils입니다
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (4 CPUs, 25.5 GB RAM, 25.5/166.8 GB disk)


In [ ]:
# 개인 구글드라이브에 데이터셋을 넣어놓았습니다
# 이걸 코랩 경로로 압축해제합니다

!unzip /content/drive/MyDrive/data2.zip -d /content

In [5]:
!mkdir /content/yolov5/data/labels

In [6]:
def get_YOLO_txt(all_json_paths):
  for json_path in all_json_paths:
    with open(json_path) as f:
      data = json.load(f)
      
      image_name = re.sub('.json', '.txt', json_path).split('/')[-1]

      dw, dh = map(float, data['FILE'][0]['RESOLUTION'].split('*'))
      dw, dh = 1/dw, 1/dh

      line = ''
      for item in data['FILE'][0]['ITEMS']:
        x1, y1, x2, y2 = map(float, item['BOX'].split(','))

        x = x1 + x2 / 2.0
        y = y1 + y2 / 2.0
        w = x2
        h = y2

        x_center = x * dw
        width = w * dw
        y_center = y * dh
        height = h * dh

        classes = 0

        line += str(classes) + ' ' + ' '.join(map(str, [x_center, y_center, width, height])) + '\n'

      line = line.rstrip()
      with open(os.path.join('/content/yolov5/data/labels', image_name), 'a') as f:
        f.write(line + '\n')

In [7]:
json_paths = glob.glob('/content/data2/label/*.json')
len(json_paths)

1636

In [8]:
real = []
for path in json_paths:
  if path.startswith('/content/data2/label/A01_B01_C04_D02_1201') or path.startswith('/content/data2/label/A01_B01_C04_D03_1122'):
    None
  else:
    real.append(path)

json_paths = real
len(json_paths)

1539

In [9]:
get_YOLO_txt(json_paths)

In [10]:
import os
import shutil

image_path = glob.glob('/content/data2/image/*.jpg')
destination_dir = '/content/yolov5/data/images'

In [11]:
# 원래 폴더 내의 모든 파일을 가져와서 목적 폴더로 이동
for path in image_path:
  if path.startswith('/content/data2/image/A01_B01_C04_D02_1201') or path.startswith('/content/data2/image/A01_B01_C04_D03_1122'):
    None
  else:
    shutil.move(path, destination_dir)

In [12]:
len(os.listdir('/content/yolov5/data/images'))

1540

In [13]:
len(os.listdir('/content/yolov5/data/labels'))

1539

In [14]:
# train, valid
images_paths = glob.glob('/content/yolov5/data/images/*.jpg')

train_path, valid_path = train_test_split(images_paths,
                                          test_size=0.2,
                                          random_state=777,
                                          shuffle=True)

with open('/content/train.txt', 'w') as f:
  f.write('\n'.join(train_path) + '\n')

with open('/content/valid.txt', 'w') as f:
  f.write('\n'.join(valid_path) + '\n')

In [15]:
import yaml

# train.txt, val.txt 파일 경로
train_file = '/content/train.txt'
val_file = '/content/valid.txt'

# 클래스 이름 리스트
classes = ['화물차']

# data.yaml 파일 경로
data_file = '/content/data.yaml'

# data.yaml 파일 생성
data = dict(
    train=train_file,
    val=val_file,
    nc=len(classes),
    names=classes
)

# 한글 문자열 지원을 위한 설정
yaml.add_representer(str, lambda dumper, data: dumper.represent_scalar('tag:yaml.org,2002:str', data, style='"'))

with open(data_file, 'w', encoding='UTF-8') as f:
    yaml.dump(data, f, allow_unicode=True)

In [16]:
!cd /content/yolov5

In [19]:
!python train.py --img 416 --batch 16 --epochs 20 --data /content/data.yaml --cfg /content/yolov5/models/yolov5m.yaml --weights yolov5m.pt --name /content/drive/MyDrive/fin_20_3

train: weights=yolov5m.pt, cfg=/content/yolov5/models/yolov5m.yaml, data=/content/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=/content/drive/MyDrive/fin_20_3, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_

In [1]:
import cv2
import torch
from tqdm import tqdm
import albumentations

def detection(input_video, output_video, model):
    cap = cv2.VideoCapture(input_video)

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

    with tqdm(total=total_frames, desc="Processing", unit="frame") as pbar:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            results = model(frame)
            result_frame = results.render()[0]
            out.write(result_frame)
            pbar.update(1)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [6]:
model = torch.hub.load('ultralytics/yolov5', 'custom', '/content/drive/MyDrive/fin_20_3/weights/best.pt')
input_video = '/content/sample4.mp4'
output_video = '/content/20_re4.mp4'

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-3-20 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5m summary: 212 layers, 20852934 parameters, 0 gradients
Adding AutoShape... 


In [7]:
detection(model = model,
          input_video = input_video,
          output_video = output_video)

Processing: 100%|██████████| 915/915 [00:43<00:00, 21.28frame/s]
